In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras as ks
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization
from keras.callbacks import EarlyStopping

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
trainX = pd.read_csv("../input/the-game-of-life-more-data/train_delta3_source.csv", index_col=False).drop(['Unnamed: 0'], axis=1)
trainX.shape[0]

In [ ]:
trainY = pd.read_csv("../input/the-game-of-life-more-data/train_delta3_target.csv", index_col=False).drop(['Unnamed: 0'], axis=1)
trainY.shape[0]

# Model

In [ ]:
def createModel():
    nn = Sequential()
    nn.add(Conv2D(256, 5, padding='same', activation='relu', input_shape=(25, 25, 1)))
    nn.add(BatchNormalization())
    for i in range(6):
        nn.add(Conv2D(256, 5, padding='same', activation='relu'))
        nn.add(BatchNormalization())
    nn.add(Conv2D(1, 5, padding='same', activation='sigmoid'))
    nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return nn
    

# Training

In [ ]:
model = createModel()
es = EarlyStopping(monitor='loss', patience=9, min_delta=0.001)
model.fit(np.array(trainX.values.reshape(-1, 25, 25, 1)), np.array(trainY.values.reshape(-1, 25, 25, 1)), batch_size=50, epochs=50, verbose=1, callbacks=[es])

In [ ]:
sampleSubmitData = pd.read_csv("../input/conways-reverse-game-of-life-2020/sample_submission.csv", index_col=0)

In [ ]:
testData = pd.read_csv("../input/conways-reverse-game-of-life-2020/test.csv", index_col=0)
testData = testData[testData.delta == 3]

In [ ]:
test = testData.iloc[:, 1:].values.reshape(-1, 25, 25, 1)
submit = model.predict(test).reshape(-1, 625).round(0).astype('uint8')
submitData = pd.DataFrame(data=submit, index=testData.index, columns=sampleSubmitData.columns)
submitData.to_csv("submit3.csv")